# Importing modules and functions

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import joblib
import pickle
from numpy import savetxt
from IPython.display import HTML
from rdkit.Chem import PandasTools

[17:47:42] Initializing Normalizer


In [2]:
def convert_smi_to_canon_smi(smi):
    
    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [3]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

# MORGAN FP_Gradient Boosting Rat oral LD50, mg/kg  

## Load data and curation work set

In [4]:
# Set file path
df_ws=pd.read_csv('rat_oral_LD50_WS.csv')
df_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCC/C=C/CC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1/C(=C\C=C2/CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


 Convert a SMILES string to canonical SMILES

In [5]:
df_ws1 = deepcopy(df_ws)
df_ws1["SMILES"] = df_ws1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ws1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


In [6]:
print('Original data: ', len(df_ws), 'molecules')
print('Failed data: ', len(df_ws1[df_ws1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  7712 molecules
Failed data:  0 molecules


In [7]:
index=df_ws1.index[df_ws1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ws.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for work set

In [8]:
df_ws1["Molecule"] = df_ws1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ws=df_ws1[df_ws1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  7712 molecules


In [9]:
moldf_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6BA...
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6BA...
2,110-54-3,CCCCCC,0.537460,24980.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6BA...
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6BA...
4,57-55-6,CC(O)CO,0.580330,19989.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6BA...
...,...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...


In [10]:
y_tr=moldf_ws.pLD50
y_tr

0       0.291207
1       0.440660
2       0.537460
3       0.539250
4       0.580330
          ...   
7707    6.514700
7708    6.937100
7709    7.099700
7710    7.206800
7711    7.602600
Name: pLD50, Length: 7712, dtype: float64

In [11]:
moldf_ws=moldf_ws.Molecule

##  Load data and curation test set

In [12]:
df_ts=pd.read_csv('rat_oral_LD50_TS.csv')
df_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)O/C(C)=C/C(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


 Convert a SMILES string to canonical SMILES

In [13]:
df_ts1 = deepcopy(df_ts)
df_ts1["SMILES"] = df_ts1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ts1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


In [14]:
print('Original data: ', len(df_ts), 'molecules')
print('Failed data: ', len(df_ts1[df_ts1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  1929 molecules
Failed data:  0 molecules


In [15]:
index=df_ts1.index[df_ts1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ts.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


##  Standardization  for test set

In [16]:
df_ts1["Molecule"] = df_ts1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ts=df_ts1[df_ts1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  1929 molecules


In [17]:
moldf_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,7782-40-3,C,0.017765,15388.8000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6B8...
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FA...
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FA...
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FA...
...,...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992,<rdkit.Chem.rdchem.Mol object at 0x000001FB6FB...


In [18]:
y_ts=moldf_ts.pLD50
y_ts

0       0.017765
1       0.572840
2       0.624490
3       0.686960
4       0.750180
          ...   
1924    5.939000
1925    6.121300
1926    6.282400
1927    6.698800
1928    9.541100
Name: pLD50, Length: 1929, dtype: float64

In [19]:
moldf_ts=moldf_ts.Molecule

## Calculation MorganFingerprint for work set

In [20]:
fp_tr = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ws]

In [21]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [22]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [23]:
savetxt('Models/MorganFingerprint/x_tr.csv', x_tr, delimiter=',')

In [24]:
x_tr.shape

(7712, 1024)

## Calculation MorganFingerprint for test set

In [25]:
fp_ts = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ts]

In [26]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [27]:
x_ts = rdkit_numpy_convert(fp_ts)

In [28]:
x_ts.shape

(1929, 1024)

In [29]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

## GradientBoostingRegressor model building and validation

In [30]:
seed = 42

In [31]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [75]:
param_grid = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }

In [76]:
m = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [77]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]},
             verbose=1)

In [78]:
m.best_params_
best_GBR = m.best_estimator_

In [79]:
m.best_params_

{'learning_rate': 0.02,
 'max_depth': 10,
 'n_estimators': 1500,
 'subsample': 0.5}

In [80]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv)

In [83]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_GBR), 2)
Q2_CV

0.55

In [82]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_GBR)), 2)
RMSE_CV

0.66

##  Prediction for test set's molecules

In [84]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [85]:
len(y_ts)

1929

In [86]:
y_pred_GBR = best_GBR.predict(x_ts)

In [87]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.57

In [88]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.64

## save the model to disk

In [89]:
pickle.dump(best_GBR, open('Models/MorganFingerprint/LD50_rat_oral_GBR_MFP.pkl', 'wb'))

## load the model from disk

In [307]:
best_GBR = pickle.load(open('Models/MorganFingerprint/LD50_rat_oral_GBR_MFP.pkl', 'rb'))

##  Y-randomization GBR model

In [163]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_GBR, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=10,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 228.7min


True score =  0.55 
Y-randomization =  -0.19 
p-value =  0.0196


[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed: 378.7min finished


##  Estimating applicability domain. Method - Euclidian distances, K=1

In [90]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [91]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [92]:
similarity= neighbors_k

In [93]:
Dmean=np.mean(similarity[1,:])

In [94]:
round(Dmean, 2)

3.63

In [95]:
std=np.std(similarity[1,:])

In [96]:
round(std, 2)

1.23

In [97]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [98]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [99]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [100]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [101]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [102]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD),2))

Coverage =  0.74


In [103]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [104]:
out_Ad=list(np.where(cpd_AD == 0)[0])

## Prediction only for molecules included in  AD

In [105]:
y_pred_GBR_ad=list(y_pred_GBR)

In [106]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [107]:
len(y_pred_GBR_ad)

1429

In [108]:
y_ts_ad=list(y_ts)

In [109]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [110]:
len(y_ts_ad)

1429

In [111]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.65

In [112]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.63

# MORGAN FP_SVM model building and validation

In [38]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [39]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [40]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=2, cv=cv, verbose=1)

In [41]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [42]:
svm.best_params_
best_svm = svm.best_estimator_

In [43]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [44]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.5

In [45]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.68

## Prediction for test set's molecules

In [46]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [47]:
y_pred_svm = best_svm.predict(x_ts)

In [48]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.55

In [50]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.66

save the model to disk

In [52]:
pickle.dump(best_svm, open('Models/MorganFingerprint/LD50_rat_oral_SVM_MF.pkl', 'wb'))

load the model from disk

In [53]:
best_svm = pickle.load(open('Models/MorganFingerprint/LD50_rat_oral_SVM_MF.pkl', 'rb'))

## Y-randomization SVM model

In [77]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_svm, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 80.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 180.5min finished


True score =  0.5 
Y-randomization =  -0.19 
p-value =  0.0099


## Estimating applicability domain. Method - Euclidian distances, K=1

In [78]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [79]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [80]:
similarity= neighbors_k

In [81]:
Dmean=np.mean(similarity[1,:])

In [82]:
round(Dmean, 2)

3.63

In [83]:
std=np.std(similarity[1,:])

In [84]:
round(std, 2)

1.23

In [85]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [86]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [87]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [88]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [89]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [90]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7407983411093831


In [91]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [92]:
out_Ad=list(np.where(cpd_AD == 0)[0])

##  Prediction only for molecules included in  AD

In [93]:
y_pred_svm_ad=list(y_pred_svm)

In [94]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [95]:
len(y_pred_svm_ad)

1429

In [96]:
y_ts_ad=list(y_ts)

In [97]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [98]:
len(y_ts_ad)

1429

In [99]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.64

In [100]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.64

# Consensus

##  load the models from disk

In [32]:
best_svm = pickle.load(open('Models/MorganFingerprint/LD50_rat_oral_SVM_MF.pkl', 'rb'))

In [33]:
best_GBR = pickle.load(open('Models/MorganFingerprint/LD50_rat_oral_GBR_MFP.pkl', 'rb'))

## Prediction for CV

In [34]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [35]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [40]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv)

In [41]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_GBR)/2

In [42]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.54

In [43]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)),2)
RMSE_CV

0.66

## Prediction for test set's molecules

In [47]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [48]:
y_pred_svm = best_svm.predict(x_ts)

In [49]:
y_pred_GBR = best_GBR.predict(x_ts)

In [50]:
y_pred_GBR

array([1.9637962 , 1.87628529, 1.75086946, ..., 3.5637255 , 3.08896394,
       2.15346429])

In [51]:
y_pred_con=(y_pred_svm+y_pred_GBR)/2

In [52]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.57

In [53]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.64

## Estimating applicability domain. Method - Euclidian distances, K=1

In [54]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [55]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,4.000000,1.414214,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,2.449490,3.316625,4.000000,4.795832,4.358899,2.645751,6.557438,3.316625,2.645751,5.567764
2,3.464102,4.690416,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,2.645751,3.464102,4.690416,5.099020,5.477226,4.472136,6.557438,4.242640,2.645751,5.916080
3,3.872983,4.795832,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,2.828427,4.000000,6.164414,6.164414,5.477226,5.099020,6.633250,5.099020,2.828427,6.244998
4,3.872983,5.099020,2.449490,4.000000,2.449490,3.464102,3.741657,6.082763,1.732051,2.645751,...,3.741657,4.242640,6.244998,6.782330,5.477226,5.099020,6.633250,7.416198,3.316625,6.324555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.630146,10.816654,10.583005,10.723805,10.246951,10.344080,10.677078,10.723805,10.677078,10.816654,...,10.954452,11.224972,11.224972,11.916375,11.357817,11.661903,11.224972,11.832160,10.677078,11.489125
7708,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.045361,11.747340,11.401754,12.206555,11.532562,11.958261,11.618950,12.247449,10.770329,11.832160
7709,11.704700,11.832160,11.747340,12.041595,11.357817,11.445523,11.401754,11.747340,11.747340,11.789826,...,12.000000,12.083046,11.832160,12.767145,12.124355,12.529964,12.206555,12.328828,11.747340,12.727922
7710,11.874342,11.916375,11.747340,12.124355,11.532562,11.532562,11.489125,11.916375,11.832160,11.958261,...,12.000000,12.165525,12.083046,13.000000,12.288206,12.688578,12.369317,12.569805,11.832160,12.884099


In [56]:
similarity= neighbors_k

In [57]:
Dmean=np.mean(similarity[1,:])

In [58]:
round(Dmean, 2)

3.63

In [59]:
std=np.std(similarity[1,:])

In [60]:
round(std, 2)

1.23

In [61]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.25


In [62]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [63]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,2.000000,3.464102,2.000000,1.414214,2.000000,2.645751,3.162278,0.000000,2.449490,4.358899,...,6.855655,4.690416,2.828427,3.741657,3.162278,4.898980,2.645751,4.123106,3.162278,7.071068
1,2.000000,3.464102,2.236068,1.732051,3.162278,2.828427,3.605551,2.449490,2.449490,4.690416,...,7.071068,6.000000,3.316625,3.872983,3.162278,5.916080,2.828427,4.358899,3.605551,7.280110
2,2.000000,3.605551,2.236068,1.732051,3.162278,2.828427,3.872983,2.449490,2.449490,4.795832,...,7.280110,6.782330,3.316625,4.795832,3.162278,6.082763,2.828427,4.582576,3.741657,7.348469
3,2.000000,3.872983,2.645751,2.000000,3.464102,2.828427,4.000000,2.449490,2.449490,5.196152,...,7.280110,6.855655,3.464102,4.795832,3.316625,6.082763,2.828427,4.690416,3.741657,7.348469
4,2.000000,4.000000,2.828427,2.236068,3.605551,2.828427,4.242640,2.449490,2.645751,5.196152,...,7.416198,6.855655,3.605551,4.898980,3.605551,6.480741,3.000000,4.690416,3.872983,7.416198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.583005,10.583005,10.295630,10.770329,10.246951,10.344080,11.000000,10.488089,10.535654,10.954452,...,11.789826,11.618950,11.401754,11.180340,11.269427,11.789826,11.357817,11.224972,10.770329,11.489125
7708,11.135529,11.135529,10.954452,11.313708,11.045361,11.000000,11.618950,11.135529,11.000000,11.224972,...,12.124355,12.124355,11.661903,11.445523,11.618950,11.874342,11.704700,11.661903,10.954452,12.083046
7709,11.575837,11.661903,11.575837,11.832160,11.575837,11.532562,11.704700,11.661903,11.618950,11.747340,...,12.449900,12.609520,12.165525,12.124355,11.958261,12.206555,12.124355,12.083046,11.832160,12.165525
7710,11.661903,11.661903,11.575837,11.916375,11.661903,11.532562,11.789826,11.661903,11.704700,12.000000,...,12.845233,12.688578,12.247449,12.124355,12.041595,12.369317,12.206555,12.247449,11.832160,12.247449


In [64]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.464 2.    ... 4.123 3.162 7.071]


In [65]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True  True False]


In [66]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7407983411093831


In [67]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1927]


In [68]:
out_Ad=list(np.where(cpd_AD == 0)[0])

## Prediction only for molecules included in  AD

In [69]:
y_pred_con_ad=list(y_pred_con)

In [70]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [71]:
len(y_pred_con_ad)

1429

In [72]:
y_ts_ad=list(y_ts)

In [73]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [74]:
len(y_ts_ad)

1429

In [75]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.65

In [76]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.63

# MACCS -RF

## Calculation MACCS Fingerprints for work set

In [148]:
from rdkit.Chem import MACCSkeys

In [149]:
fp_tr = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ws]

In [152]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [153]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [154]:
savetxt('Models/MACCS/x_tr_MACCS.csv', x_tr, delimiter=',')

In [155]:
x_tr.shape

(7712, 167)

## Calculation  MACCS Fingerprint for test set

In [156]:
fp_ts = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ts]

In [157]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [158]:
x_ts = rdkit_numpy_convert(fp_ts)

In [159]:
x_ts.shape

(1929, 167)

In [160]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

##  Random forest model building and validation¶

In [161]:
seed = 42

In [162]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [163]:
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3, x_tr.shape[1] // 2],
              "n_estimators": [100, 250, 500, 1000]}

In [164]:
m = GridSearchCV(RandomForestRegressor(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [165]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestRegressor(), n_jobs=2,
             param_grid={'max_features': [16, 23, 33, 55, 83],
                         'n_estimators': [100, 250, 500, 1000]},
             verbose=1)

In [166]:
m.best_params_
best_RF = m.best_estimator_

In [167]:
m.best_params_

{'max_features': 33, 'n_estimators': 1000}

In [168]:
y_pred_CV_RF = cross_val_predict(best_RF, x_tr, y_tr, cv=cv)

In [171]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_RF), 2)
Q2_CV

0.58

In [172]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_RF)), 2)
RMSE_CV

0.65

##  Prediction for test set's molecules

In [173]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [174]:
len(y_ts)

1929

In [175]:
y_pred_rf = best_RF.predict(x_ts)

In [176]:
Q2_TS = round(r2_score(y_ts, y_pred_rf), 2)
Q2_TS

0.59

In [177]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_rf)), 2)
RMSE_TS

0.63

## save the model to disk

In [179]:
pickle.dump(best_RF, open('Models/MACCS/LD50_rat_oral_RF_MACCS.pkl', 'wb'))

## load the model from disk

In [180]:
best_RF = pickle.load(open('Models/MACCS/LD50_rat_oral_RF_MACCS.pkl', 'rb'))

##  Y-randomization RF model

In [204]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_RF, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 26.5min finished


True score =  0.58 
Y-randomization =  -0.13 
p-value =  0.0196


##  Estimating applicability domain. Method - Euclidian distances, K=1

In [205]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [206]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.000000,4.000000,1.414214,2.236068,2.000000,2.645751,3.000000,2.645751,0.000000,0.000000,...,0.000000,1.000000,1.732051,3.162278,2.645751,2.000000,3.162278,2.449490,0.000000,3.000000
2,1.732051,4.242640,1.414214,2.236068,2.000000,2.828427,3.162278,2.828427,0.000000,0.000000,...,0.000000,2.449490,2.236068,3.162278,3.741657,2.449490,3.162278,2.449490,0.000000,3.162278
3,2.449490,4.358899,1.732051,2.449490,2.449490,3.162278,3.162278,3.000000,0.000000,0.000000,...,0.000000,3.000000,4.242640,3.464102,5.000000,2.449490,3.316625,3.000000,0.000000,3.162278
4,2.645751,4.690416,1.732051,2.645751,2.449490,3.162278,3.162278,3.000000,0.000000,0.000000,...,0.000000,3.162278,4.358899,3.464102,5.000000,2.449490,3.316625,3.464102,0.000000,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,8.246211,8.602325,9.219544,8.944272,9.165152,8.888194,9.000000,9.380832,9.110434,9.327379,...,9.000000,8.774964,8.544003,8.774964,8.306623,9.000000,8.774964,9.110434,9.000000,8.544003
7708,8.246211,8.602325,9.273619,9.000000,9.219544,8.888194,9.000000,9.380832,9.165152,9.327379,...,9.055386,8.831760,8.544003,8.774964,8.366600,9.000000,8.831760,9.110434,9.055386,8.544003
7709,8.306623,8.602325,9.327379,9.000000,9.219544,8.944272,9.000000,9.486833,9.219544,9.539392,...,9.110434,8.831760,8.602325,8.774964,8.426149,9.000000,8.831760,9.219544,9.110434,8.602325
7710,8.485281,8.717798,9.380832,9.000000,9.219544,8.944272,9.055386,9.539392,9.273619,9.539392,...,9.219544,8.888194,8.602325,8.888194,8.426149,9.000000,8.944272,9.273619,9.219544,8.774964


In [207]:
similarity= neighbors_k

In [208]:
Dmean=np.mean(similarity[1,:])

In [209]:
round(Dmean, 2)

2.03

In [210]:
std=np.std(similarity[1,:])

In [211]:
round(std, 2)

1.11

In [212]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.59


In [213]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [214]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,1.414214,2.236068,3.000000,0.000000,2.449490,2.000000,0.000000,1.414214,2.000000,2.236068,...,2.236068,2.000000,0.000000,3.162278,1.000000,0.000000,0.000000,2.449490,1.414214,2.236068
1,1.732051,2.828427,3.162278,0.000000,2.449490,2.000000,0.000000,1.732051,2.000000,2.828427,...,2.645751,2.645751,0.000000,3.605551,1.414214,1.732051,0.000000,2.449490,1.414214,2.449490
2,1.732051,3.000000,3.162278,0.000000,3.162278,2.000000,0.000000,1.732051,2.449490,2.828427,...,2.828427,3.316625,0.000000,3.605551,2.236068,2.449490,0.000000,2.645751,1.414214,2.449490
3,1.732051,3.162278,3.316625,0.000000,3.872983,2.000000,1.414214,1.732051,2.645751,2.828427,...,3.000000,3.605551,0.000000,3.605551,2.449490,2.645751,0.000000,2.645751,1.414214,2.645751
4,1.732051,3.162278,3.316625,0.000000,4.000000,2.000000,1.414214,2.000000,2.828427,2.828427,...,3.000000,4.000000,1.000000,3.741657,2.449490,2.828427,1.000000,2.645751,1.414214,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,9.165152,9.110434,8.660254,9.110434,8.602325,9.273619,9.110434,9.165152,9.110434,8.831760,...,8.944272,8.944272,8.660254,8.774964,9.000000,9.055386,8.660254,8.660254,9.110434,9.055386
7708,9.273619,9.165152,8.717798,9.165152,8.602325,9.273619,9.110434,9.219544,9.110434,8.888194,...,9.000000,9.110434,8.717798,8.774964,9.000000,9.055386,8.717798,8.660254,9.110434,9.055386
7709,9.380832,9.219544,8.717798,9.219544,8.660254,9.327379,9.327379,9.219544,9.165152,8.944272,...,9.055386,9.110434,8.831760,8.774964,9.055386,9.110434,8.831760,8.660254,9.110434,9.110434
7710,9.380832,9.273619,8.831760,9.273619,8.831760,9.486833,9.327379,9.327379,9.219544,9.000000,...,9.273619,9.110434,8.944272,8.774964,9.055386,9.165152,8.944272,8.660254,9.219544,9.219544


In [215]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.414 2.236 3.    ... 2.449 1.414 2.236]


In [216]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False ...  True  True  True]


In [217]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD),2))

Coverage =  0.69


In [218]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    3 ... 1926 1927 1928]


In [219]:
out_Ad=list(np.where(cpd_AD == 0)[0])

## Prediction only for molecules included in  AD

In [220]:
y_pred_rf_ad=list(y_pred_rf)

In [221]:
y_pred_rf_ad[:] = [x for i,x in enumerate(y_pred_rf_ad) if i not in out_Ad]

In [222]:
len(y_pred_rf_ad)

1332

In [223]:
y_ts_ad=list(y_ts)

In [224]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [225]:
len(y_ts_ad)

1332

In [226]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_rf_ad), 2)
Q2_TS

0.65

In [227]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_rf_ad)), 2)
RMSE_TS

0.61

# MACCS FP_SVM model building and validation

In [228]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [229]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [230]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=2, cv=cv, verbose=1)

In [231]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [232]:
svm.best_params_
best_svm = svm.best_estimator_

In [233]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [234]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.56

In [235]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.65

## Prediction for test set's molecules

In [236]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [237]:
y_pred_svm = best_svm.predict(x_ts)

In [238]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.58

In [239]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.64

save the model to disk

In [240]:
pickle.dump(best_svm, open('Models/MACCS/LD50_rat_oral_SVM_MACCS.pkl', 'wb'))

load the model from disk

In [241]:
best_svm = pickle.load(open('Models/MACCS/LD50_rat_oral_SVM_MACCS.pkl', 'rb'))

## Y-randomization SVM model

In [242]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_svm, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 13.2min


True score =  0.56 
Y-randomization =  -0.1 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 15.2min finished


## Estimating applicability domain. Method - Euclidian distances, K=1

In [243]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [244]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.000000,4.000000,1.414214,2.236068,2.000000,2.645751,3.000000,2.645751,0.000000,0.000000,...,0.000000,1.000000,1.732051,3.162278,2.645751,2.000000,3.162278,2.449490,0.000000,3.000000
2,1.732051,4.242640,1.414214,2.236068,2.000000,2.828427,3.162278,2.828427,0.000000,0.000000,...,0.000000,2.449490,2.236068,3.162278,3.741657,2.449490,3.162278,2.449490,0.000000,3.162278
3,2.449490,4.358899,1.732051,2.449490,2.449490,3.162278,3.162278,3.000000,0.000000,0.000000,...,0.000000,3.000000,4.242640,3.464102,5.000000,2.449490,3.316625,3.000000,0.000000,3.162278
4,2.645751,4.690416,1.732051,2.645751,2.449490,3.162278,3.162278,3.000000,0.000000,0.000000,...,0.000000,3.162278,4.358899,3.464102,5.000000,2.449490,3.316625,3.464102,0.000000,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,8.246211,8.602325,9.219544,8.944272,9.165152,8.888194,9.000000,9.380832,9.110434,9.327379,...,9.000000,8.774964,8.544003,8.774964,8.306623,9.000000,8.774964,9.110434,9.000000,8.544003
7708,8.246211,8.602325,9.273619,9.000000,9.219544,8.888194,9.000000,9.380832,9.165152,9.327379,...,9.055386,8.831760,8.544003,8.774964,8.366600,9.000000,8.831760,9.110434,9.055386,8.544003
7709,8.306623,8.602325,9.327379,9.000000,9.219544,8.944272,9.000000,9.486833,9.219544,9.539392,...,9.110434,8.831760,8.602325,8.774964,8.426149,9.000000,8.831760,9.219544,9.110434,8.602325
7710,8.485281,8.717798,9.380832,9.000000,9.219544,8.944272,9.055386,9.539392,9.273619,9.539392,...,9.219544,8.888194,8.602325,8.888194,8.426149,9.000000,8.944272,9.273619,9.219544,8.774964


In [245]:
similarity= neighbors_k

In [246]:
Dmean=np.mean(similarity[1,:])

In [247]:
round(Dmean, 2)

2.03

In [248]:
std=np.std(similarity[1,:])

In [249]:
round(std, 2)

1.11

In [250]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.59


In [251]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [252]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,1.414214,2.236068,3.000000,0.000000,2.449490,2.000000,0.000000,1.414214,2.000000,2.236068,...,2.236068,2.000000,0.000000,3.162278,1.000000,0.000000,0.000000,2.449490,1.414214,2.236068
1,1.732051,2.828427,3.162278,0.000000,2.449490,2.000000,0.000000,1.732051,2.000000,2.828427,...,2.645751,2.645751,0.000000,3.605551,1.414214,1.732051,0.000000,2.449490,1.414214,2.449490
2,1.732051,3.000000,3.162278,0.000000,3.162278,2.000000,0.000000,1.732051,2.449490,2.828427,...,2.828427,3.316625,0.000000,3.605551,2.236068,2.449490,0.000000,2.645751,1.414214,2.449490
3,1.732051,3.162278,3.316625,0.000000,3.872983,2.000000,1.414214,1.732051,2.645751,2.828427,...,3.000000,3.605551,0.000000,3.605551,2.449490,2.645751,0.000000,2.645751,1.414214,2.645751
4,1.732051,3.162278,3.316625,0.000000,4.000000,2.000000,1.414214,2.000000,2.828427,2.828427,...,3.000000,4.000000,1.000000,3.741657,2.449490,2.828427,1.000000,2.645751,1.414214,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,9.165152,9.110434,8.660254,9.110434,8.602325,9.273619,9.110434,9.165152,9.110434,8.831760,...,8.944272,8.944272,8.660254,8.774964,9.000000,9.055386,8.660254,8.660254,9.110434,9.055386
7708,9.273619,9.165152,8.717798,9.165152,8.602325,9.273619,9.110434,9.219544,9.110434,8.888194,...,9.000000,9.110434,8.717798,8.774964,9.000000,9.055386,8.717798,8.660254,9.110434,9.055386
7709,9.380832,9.219544,8.717798,9.219544,8.660254,9.327379,9.327379,9.219544,9.165152,8.944272,...,9.055386,9.110434,8.831760,8.774964,9.055386,9.110434,8.831760,8.660254,9.110434,9.110434
7710,9.380832,9.273619,8.831760,9.273619,8.831760,9.486833,9.327379,9.327379,9.219544,9.000000,...,9.273619,9.110434,8.944272,8.774964,9.055386,9.165152,8.944272,8.660254,9.219544,9.219544


In [253]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.414 2.236 3.    ... 2.449 1.414 2.236]


In [254]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False ...  True  True  True]


In [255]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.6905132192846034


In [256]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    3 ... 1926 1927 1928]


In [257]:
out_Ad=list(np.where(cpd_AD == 0)[0])

##  Prediction only for molecules included in  AD

In [258]:
y_pred_svm_ad=list(y_pred_svm)

In [259]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [260]:
len(y_pred_svm_ad)

1332

In [261]:
y_ts_ad=list(y_ts)

In [262]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [263]:
len(y_ts_ad)

1332

In [264]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.64

In [265]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.62

# Consensus

##  load the models from disk

In [266]:
best_svm = pickle.load(open('Models/MACCS/LD50_rat_oral_SVM_MACCS.pkl', 'rb'))

In [267]:
best_rf = pickle.load(open('Models/MACCS/LD50_rat_oral_RF_MACCS.pkl', 'rb'))

## Prediction for CV

In [268]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [269]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [270]:
y_pred_CV_rf = cross_val_predict(best_rf, x_tr, y_tr, cv=cv)

In [271]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_rf)/2

In [272]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.58

In [273]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)),2)
RMSE_CV

0.65

## Prediction for test set's molecules

In [274]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [275]:
y_pred_svm = best_svm.predict(x_ts)

In [276]:
y_pred_rf = best_rf.predict(x_ts)

In [277]:
y_pred_rf

array([1.72010343, 1.88733118, 1.6128627 , ..., 3.55339399, 5.29791271,
       2.28612094])

In [278]:
y_pred_con=(y_pred_svm+y_pred_rf)/2

In [279]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.59

In [280]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.63

## Estimating applicability domain. Method - Euclidian distances, K=1

In [281]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [282]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.000000,4.000000,1.414214,2.236068,2.000000,2.645751,3.000000,2.645751,0.000000,0.000000,...,0.000000,1.000000,1.732051,3.162278,2.645751,2.000000,3.162278,2.449490,0.000000,3.000000
2,1.732051,4.242640,1.414214,2.236068,2.000000,2.828427,3.162278,2.828427,0.000000,0.000000,...,0.000000,2.449490,2.236068,3.162278,3.741657,2.449490,3.162278,2.449490,0.000000,3.162278
3,2.449490,4.358899,1.732051,2.449490,2.449490,3.162278,3.162278,3.000000,0.000000,0.000000,...,0.000000,3.000000,4.242640,3.464102,5.000000,2.449490,3.316625,3.000000,0.000000,3.162278
4,2.645751,4.690416,1.732051,2.645751,2.449490,3.162278,3.162278,3.000000,0.000000,0.000000,...,0.000000,3.162278,4.358899,3.464102,5.000000,2.449490,3.316625,3.464102,0.000000,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,8.246211,8.602325,9.219544,8.944272,9.165152,8.888194,9.000000,9.380832,9.110434,9.327379,...,9.000000,8.774964,8.544003,8.774964,8.306623,9.000000,8.774964,9.110434,9.000000,8.544003
7708,8.246211,8.602325,9.273619,9.000000,9.219544,8.888194,9.000000,9.380832,9.165152,9.327379,...,9.055386,8.831760,8.544003,8.774964,8.366600,9.000000,8.831760,9.110434,9.055386,8.544003
7709,8.306623,8.602325,9.327379,9.000000,9.219544,8.944272,9.000000,9.486833,9.219544,9.539392,...,9.110434,8.831760,8.602325,8.774964,8.426149,9.000000,8.831760,9.219544,9.110434,8.602325
7710,8.485281,8.717798,9.380832,9.000000,9.219544,8.944272,9.055386,9.539392,9.273619,9.539392,...,9.219544,8.888194,8.602325,8.888194,8.426149,9.000000,8.944272,9.273619,9.219544,8.774964


In [283]:
similarity= neighbors_k

In [284]:
Dmean=np.mean(similarity[1,:])

In [285]:
round(Dmean, 2)

2.03

In [286]:
std=np.std(similarity[1,:])

In [287]:
round(std, 2)

1.11

In [288]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.59


In [289]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [290]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,1.414214,2.236068,3.000000,0.000000,2.449490,2.000000,0.000000,1.414214,2.000000,2.236068,...,2.236068,2.000000,0.000000,3.162278,1.000000,0.000000,0.000000,2.449490,1.414214,2.236068
1,1.732051,2.828427,3.162278,0.000000,2.449490,2.000000,0.000000,1.732051,2.000000,2.828427,...,2.645751,2.645751,0.000000,3.605551,1.414214,1.732051,0.000000,2.449490,1.414214,2.449490
2,1.732051,3.000000,3.162278,0.000000,3.162278,2.000000,0.000000,1.732051,2.449490,2.828427,...,2.828427,3.316625,0.000000,3.605551,2.236068,2.449490,0.000000,2.645751,1.414214,2.449490
3,1.732051,3.162278,3.316625,0.000000,3.872983,2.000000,1.414214,1.732051,2.645751,2.828427,...,3.000000,3.605551,0.000000,3.605551,2.449490,2.645751,0.000000,2.645751,1.414214,2.645751
4,1.732051,3.162278,3.316625,0.000000,4.000000,2.000000,1.414214,2.000000,2.828427,2.828427,...,3.000000,4.000000,1.000000,3.741657,2.449490,2.828427,1.000000,2.645751,1.414214,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,9.165152,9.110434,8.660254,9.110434,8.602325,9.273619,9.110434,9.165152,9.110434,8.831760,...,8.944272,8.944272,8.660254,8.774964,9.000000,9.055386,8.660254,8.660254,9.110434,9.055386
7708,9.273619,9.165152,8.717798,9.165152,8.602325,9.273619,9.110434,9.219544,9.110434,8.888194,...,9.000000,9.110434,8.717798,8.774964,9.000000,9.055386,8.717798,8.660254,9.110434,9.055386
7709,9.380832,9.219544,8.717798,9.219544,8.660254,9.327379,9.327379,9.219544,9.165152,8.944272,...,9.055386,9.110434,8.831760,8.774964,9.055386,9.110434,8.831760,8.660254,9.110434,9.110434
7710,9.380832,9.273619,8.831760,9.273619,8.831760,9.486833,9.327379,9.327379,9.219544,9.000000,...,9.273619,9.110434,8.944272,8.774964,9.055386,9.165152,8.944272,8.660254,9.219544,9.219544


In [291]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.414 2.236 3.    ... 2.449 1.414 2.236]


In [292]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False ...  True  True  True]


In [293]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.6905132192846034


In [294]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    3 ... 1926 1927 1928]


In [295]:
out_Ad=list(np.where(cpd_AD == 0)[0])

## Prediction only for molecules included in  AD

In [296]:
y_pred_con_ad=list(y_pred_con)

In [297]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [298]:
len(y_pred_con_ad)

1332

In [299]:
y_ts_ad=list(y_ts)

In [300]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [301]:
len(y_ts_ad)

1332

In [302]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.65

In [303]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.61